In [1]:
import warnings
warnings.filterwarnings("ignore", message="The PyTorch API of nested tensors is in prototype stage and will change in the near future.")
# warnings.filterwarnings("ignore", message="The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible.")
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from CustomDataCollatorForSequenceClassification import CustomDataCollatorForSequenceClassification
from torch.optim import AdamW
from datasets import Dataset
import random
import pandas as pd
import numpy as np
from WordPieceTokenizer import WordPieceTokenizer as Tokenizer
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
from CustomBertSequenceClassification import CustomBertSequenceClassification
from CustomBert import CustomBertConfig
import CustomBert
from collections import Counter
import os
from Model import LSTM
from Model import Transformer, PositionalEncoding
from sklearn.metrics import f1_score
from torch.optim.lr_scheduler import _LRScheduler,ReduceLROnPlateau
from torch.utils.tensorboard import SummaryWriter
from TrainDataset import prepare_classification_dataset, tensor_dataset
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

dataFilePath = 'datasets/'
saveFilePath = 'saves/'
vocab_file_path = f'{saveFilePath}vocab.txt'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = Tokenizer(vocab_file_path,do_lower_case=False,strip_accents=False,clean_text=True)
VOCAB_SIZE = tokenizer.get_vocab_size()
MAX_SEQUENCE_LENGTH = 64
BATCH_SIZE = 32

In [2]:
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

# SEED = 32
# set_seed(SEED)

In [3]:
df = pd.read_csv(f'datasets/sentiment_train.csv', index_col=0)

# '감정' 레이블을 숫자로 매핑
df.loc[(df['감정'] == '불안'), '감정'] = 0
df.loc[(df['감정'] == '당황'), '감정'] = 1
df.loc[(df['감정'] == '분노'), '감정'] = 2
df.loc[(df['감정'] == '슬픔'), '감정'] = 3
df.loc[(df['감정'] == '중립'), '감정'] = 4
df.loc[(df['감정'] == '행복'), '감정'] = 5
df.loc[(df['감정'] == '혐오'), '감정'] = 6

print(f"원본 df 크기: {len(df)}")
print(f"원본 df 감정 분포 (매핑 후): {Counter(df['감정'])}")

train_df, val_df = train_test_split(df, train_size=0.8, test_size=0.2, stratify=df['감정'], random_state=42) # 재현성을 위해 random_state 추가

print(f"학습 데이터프레임 크기: {len(train_df)}")
print(f"검증 데이터프레임 크기: {len(val_df)}")
print(f"학습 데이터프레임 감정 분포: {Counter(train_df['감정'])}")
print(f"검증 데이터프레임 감정 분포: {Counter(val_df['감정'])}")


print("학습 데이터 파싱 중...")
train_datasets_dict = prepare_classification_dataset(train_df)
print("검증 데이터 파싱 중...")
val_datasets_dict = prepare_classification_dataset(val_df)

train_datasets = tensor_dataset(train_datasets_dict)
val_datasets = tensor_dataset(val_datasets_dict)

print(f"학습 데이터셋 크기: {len(train_datasets)}")
print(f"검증 데이터셋 크기: {len(val_datasets)}")

train_loader = DataLoader(
    train_datasets,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=0
)
print(f"학습 DataLoader 배치 수: {len(train_loader)}")

val_loader = DataLoader(
    val_datasets,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0
)
print(f"검증 DataLoader 배치 수: {len(val_loader)}")

원본 df 크기: 32194
원본 df 감정 분포 (매핑 후): Counter({5: 5948, 2: 5580, 0: 5414, 6: 5259, 3: 5223, 4: 4770})
학습 데이터프레임 크기: 25755
검증 데이터프레임 크기: 6439
학습 데이터프레임 감정 분포: Counter({5: 4758, 2: 4464, 0: 4331, 6: 4207, 3: 4179, 4: 3816})
검증 데이터프레임 감정 분포: Counter({5: 1190, 2: 1116, 0: 1083, 6: 1052, 3: 1044, 4: 954})
학습 데이터 파싱 중...


데이터 파싱 중:   0%|          | 0/25755 [00:00<?, ?it/s]

검증 데이터 파싱 중...


데이터 파싱 중:   0%|          | 0/6439 [00:00<?, ?it/s]

학습 데이터셋 크기: 25755
검증 데이터셋 크기: 6439
학습 DataLoader 배치 수: 805
검증 DataLoader 배치 수: 202


## LSTM

In [4]:
# def process_dataframe(data_frame, device,batch_size,shuffle=False):
#     tensor_x_list = []
#     attentions = []
#     token_type_ids_ = []
#     for i in tqdm(range(len(data_frame))):
#         token = data_frame.iloc[i,0]
#         token = token.split(" ")
#         token_list = []
#         for t in token:
#             token_list.append(int(t))
#         tensor_x_list.append(token_list)
        
#         attention = data_frame.iloc[i,3]
#         attention = attention.split(" ")
#         attention_list = []
#         for a in attention:
#             attention_list.append(int(a))
#         attentions.append(attention_list)

#         token_type_ids = data_frame.iloc[i,4]
#         token_type_ids = token_type_ids.split(" ")
#         token_type_ids_list = []
#         for t in token_type_ids:
#             token_type_ids_list.append(int(t))
#         token_type_ids_.append(attention_list)
        
#     tensor_x = torch.tensor(tensor_x_list, dtype=torch.long, device=device)
#     tensor_attention = torch.tensor(attentions, dtype=torch.long, device=device)
#     tensor_token_type_ids = torch.tensor(token_type_ids_, dtype=torch.long, device=device)
#     tensor_t = torch.tensor(data_frame["감정"].values.tolist(), dtype=torch.long, device=device)

#     dataset = TensorDataset(tensor_x,tensor_attention,tensor_t,tensor_token_type_ids)
#     loader = DataLoader(dataset,batch_size=batch_size,shuffle=shuffle,drop_last=True)
#     return loader
    
#     dataset = {"input_ids" : tensor_x, "attention_mask":tensor_attention,"token_type_ids":tensor_token_type_ids,"labels":tensor_t}
    
    

#     data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [5]:
def LSTM_Train(epoch,device,train_loader,val_loader,NN,loss_function,optimizer):
    acc = 0
    prev_acc = 0
    cnt = 0
    for e in range(epoch):
        NN.to(device)
        loss_sum = 0
        NN.train()
        for x, attention,t in train_loader:
            y = NN(x,attention)
            loss = loss_function(y,t)
            loss_sum += loss.item()
    
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        loss_sum /= len(train_loader)
    
        NN.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for x, attention, t in val_loader:
                x = x.to(device)
                attention = attention.to(device)
                t = t.to(device)
    
                y = NN(x, attention)
                correct += (y.argmax(dim=-1) == t).sum().item()
                total += len(x)
        acc = correct / total
    
        if acc <= prev_acc:
            cnt += 1
        else :
            torch.save(NN.state_dict(), "Sentiment.pt")
            cnt = 0
            prev_acc = acc
        
        print(f"epoch  {e+1}\t\tloss {loss_sum:.12f}\tacc {acc:.4f}\tcnt {cnt}")
        
        if cnt >= 5:
            print("train halted")
            break
            
    print("---------- 학습 종료 ----------")

In [6]:
# NN = LSTM(vocab_size=vocab_size,embedding_dim=embedding_dim,hidden_dim=64,output_dim=7,n_layers=4,bidirectional=True,dropout_p=0.1)
# NN.to(device)
# loss_function = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(NN.parameters(),lr=0.001)
# epoch = 500
# LSTM_Train(epoch,device,train_loader,val_loader,NN,loss_function,optimizer)

## Transformer

In [7]:
class GradualWarmupScheduler(_LRScheduler):
    def __init__(self, optimizer, multiplier, total_epoch, after_scheduler=None):
        self.multiplier = multiplier
        self.total_epoch = total_epoch
        self.after_scheduler = after_scheduler
        self.finished = False
        super().__init__(optimizer)

    def get_lr(self):
        if self.last_epoch > self.total_epoch:
            if self.after_scheduler:
                if not self.finished:
                    self.after_scheduler.base_lrs = self.base_lrs
                    self.finished = True
                return self.after_scheduler.get_last_lr()
            return [base_lr * self.multiplier for base_lr in self.base_lrs]

        return [base_lr * ((self.multiplier - 1.) * self.last_epoch / self.total_epoch + 1.) for base_lr in self.base_lrs]

    def step(self, metrics=None):
        if self.finished and self.after_scheduler:
            if isinstance(self.after_scheduler, ReduceLROnPlateau) and metrics is not None:
                return self.after_scheduler.step(metrics)
            else:
                return self.after_scheduler.step() # metrics가 없으면 파라미터 없이 호출 (일반 스케줄러)
        else:
            return super(GradualWarmupScheduler, self).step() # Warmup 기간에는 파라미터 없이 호출

def Transformer_Train(epoch, device, train_loader, val_loader, NN, loss_function, optimizer, scheduler,
                      warmup_epochs=5, log_dir="runs/sentiment_experiment", 
                      save_path="saves/models/Sentiment.pt", multiplier=1.0, patience=5):
    
    writer = SummaryWriter(log_dir)

    combined_scheduler = GradualWarmupScheduler(optimizer, multiplier=multiplier, total_epoch=warmup_epochs, after_scheduler=scheduler)
    
    best_f1_weighted = 0.0
    epochs_no_improve = 0
    
    for e in range(epoch):
        NN.to(device)
        
        train_loss_sum = 0
        NN.train()
        for x, attention, t in tqdm(train_loader, desc=f"Epoch {e+1} Training"):
            x = x.to(device)
            attention = attention.to(device)
            t = t.to(device)

            y = NN(x, attention)
            loss = loss_function(y, t)
            train_loss_sum += loss.item()
            
            optimizer.zero_grad()
            loss.backward()
            nn.utils.clip_grad_norm_(NN.parameters(), 1.0)
            optimizer.step()
        train_loss_sum /= len(train_loader)
        
        NN.eval()
        val_correct = 0
        val_total = 0
        val_all_preds = []
        val_all_targets = []
        with torch.no_grad():
            for x, attention, t in tqdm(val_loader, desc=f"Epoch {e+1} Validation", leave=False):
                x = x.to(device)
                attention = attention.to(device)
                t = t.to(device)
                
                y = NN(x, attention)
                
                preds = y.argmax(dim=-1)
                val_correct += (preds == t).sum().item()
                val_total += len(x)

                val_all_preds.extend(preds.cpu().numpy())
                val_all_targets.extend(t.cpu().numpy())
        
        val_acc = val_correct / val_total
        
        if len(np.unique(val_all_targets)) > 1:
            val_f1_weighted = f1_score(val_all_targets, val_all_preds, average='weighted', zero_division=0)
            val_f1_macro = f1_score(val_all_targets, val_all_preds, average='macro', zero_division=0)
        else:
            val_f1_weighted = 1.0 if (len(val_all_targets) > 0 and np.all(np.array(val_all_preds) == np.array(val_all_targets))) else 0.0
            val_f1_macro = val_f1_weighted

        if val_f1_weighted > best_f1_weighted:
            best_f1_weighted = val_f1_weighted
            torch.save(NN.state_dict(), save_path)
            print(f"모델 저장 완료 (Best Weighted F1: {best_f1_weighted:.4f}).")
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        # 스케줄러 스텝: val_f1_weighted 값을 'metrics' 파라미터로 전달
        combined_scheduler.step(val_f1_weighted)
        
        # TensorBoard에 로깅
        writer.add_scalar('Loss/train', train_loss_sum, e)
        writer.add_scalar('Metrics/val_accuracy', val_acc, e)
        writer.add_scalar('Metrics/val_f1_weighted', val_f1_weighted, e)
        writer.add_scalar('Metrics/val_f1_macro', val_f1_macro, e)
        writer.add_scalar('LearningRate', optimizer.param_groups[0]['lr'], e)
        
        print(f"Epoch {e+1}\tTrain Loss: {train_loss_sum:.6f}\tVal Acc: {val_acc:.4f}\t\tVal F1 (Weighted): {val_f1_weighted:.4f}\tVal F1 (Macro): {val_f1_macro:.4f}\tNo Improve Epochs: {epochs_no_improve}")
        
        if epochs_no_improve >= patience:
            print("조기 종료: 검증 F1 점수 개선 없음.")
            break
            
    writer.close()
    print("---------- 학습 종료 ----------")

In [8]:
train_number = 24
save_path = f"saves/models/Sentiment_v{train_number}.pt"
NN = Transformer(vocab_size=VOCAB_SIZE,embedding_dim=256,hidden_dim=128,output_dim=7,n_layers=4,
                 n_heads=8,dropout_p=0.4,max_len=128,pad_token_id=0)
# NN.load_state_dict(torch.load("saves/models/Sentiment_v19.pt"))
all_train_labels_original = train_df['감정'].values.astype(int)

num_classes = NN.output_dim

label_counts_original = np.bincount(all_train_labels_original, minlength=num_classes)
class_counts_tensor = torch.tensor(label_counts_original, dtype=torch.float)

class_counts_tensor = torch.where(class_counts_tensor == 0, torch.tensor(1.0), class_counts_tensor)

print(f"학습 데이터 클래스별 샘플 수: {class_counts_tensor.tolist()}")
print(f"학습 데이터 클래스 분포: {Counter(all_train_labels_original)}")

class_weights = (class_counts_tensor.sum() / class_counts_tensor)

class_weights = class_weights.to(device)

print(f"계산된 클래스 가중치: {class_weights.tolist()}")

# loss_function = nn.CrossEntropyLoss(weight=class_weights)
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(NN.parameters(), lr=4.565359362828951e-05)
scheduler_plateau = ReduceLROnPlateau(optimizer, mode="max", factor=0.8, patience=5,min_lr=1e-7)
epoch = 10000

data, attention_mask, labels = next(iter(train_loader))
print(f"첫 배치 레이블 분포: {Counter(labels.cpu().numpy())}")

Transformer_Train(epoch, device, train_loader, val_loader, NN, loss_function, optimizer, scheduler_plateau,
                  warmup_epochs=5,
                  log_dir=f"runs/sentiment_experiment_v{train_number}",
                  save_path=save_path,
                  multiplier=1.0,
                  patience=20)

학습 데이터 클래스별 샘플 수: [4331.0, 1.0, 4464.0, 4179.0, 3816.0, 4758.0, 4207.0]
학습 데이터 클래스 분포: Counter({np.int64(5): 4758, np.int64(2): 4464, np.int64(0): 4331, np.int64(6): 4207, np.int64(3): 4179, np.int64(4): 3816})
계산된 클래스 가중치: [5.946894645690918, 25756.0, 5.769713401794434, 6.163197040557861, 6.749475955963135, 5.413198947906494, 6.1221771240234375]
첫 배치 레이블 분포: Counter({np.int64(4): 7, np.int64(6): 6, np.int64(2): 6, np.int64(3): 5, np.int64(5): 4, np.int64(0): 4})


Epoch 1 Training:   0%|          | 0/805 [00:00<?, ?it/s]

Epoch 1 Validation:   0%|          | 0/202 [00:00<?, ?it/s]

모델 저장 완료 (Best Weighted F1: 0.1575).
Epoch 1	Train Loss: 1.894305	Val Acc: 0.2194		Val F1 (Weighted): 0.1575	Val F1 (Macro): 0.1532	No Improve Epochs: 0


Epoch 2 Training:   0%|          | 0/805 [00:00<?, ?it/s]

...

Epoch 134 Validation:   0%|          | 0/202 [00:00<?, ?it/s]

Epoch 134	Train Loss: 1.100763	Val Acc: 0.4920		Val F1 (Weighted): 0.4947	Val F1 (Macro): 0.4878	No Improve Epochs: 19


Epoch 135 Training:   0%|          | 0/805 [00:00<?, ?it/s]

Epoch 135 Validation:   0%|          | 0/202 [00:00<?, ?it/s]

Epoch 135	Train Loss: 1.099477	Val Acc: 0.4937		Val F1 (Weighted): 0.4962	Val F1 (Macro): 0.4893	No Improve Epochs: 20
조기 종료: 검증 F1 점수 개선 없음.
---------- 학습 종료 ----------


# Transfer Model

PRETRAIN_MODEL_SAVE_PATH = "saves/Pretrain.pt"
MODEL_SAVE_PATH = "Sentiment.pt"
HIDDEN_SIZE = 768
NUM_HIDDEN_LAYERS = 12
NUM_ATTENTION_HEADS = 12
INTERMEDIATE_SIZE = 3072
TYPE_VOCAB_SIZE = 2
DROPOUT_PROB = 0.1

config = CustomBertConfig(
    VOCAB_SIZE=VOCAB_SIZE,
    HIDDEN_SIZE=HIDDEN_SIZE,
    NUM_HIDDEN_LAYERS=NUM_HIDDEN_LAYERS,
    NUM_ATTENTION_HEADS=NUM_ATTENTION_HEADS,
    INTERMEDIATE_SIZE=INTERMEDIATE_SIZE,
    MAX_SEQUENCE_LENGTH=MAX_SEQUENCE_LENGTH,
    TYPE_VOCAB_SIZE=TYPE_VOCAB_SIZE,
    DROPOUT_PROB=DROPOUT_PROB
)

model = CustomBertSequenceClassification(config,PRETRAIN_MODEL_SAVE_PATH,7)

if os.path.exists(MODEL_SAVE_PATH):
    print("모델 가중치 로드 중...")
    # 먼저 CPU에 로드한 후 모델에 로드합니다.
    loaded_state_dict = torch.load(MODEL_SAVE_PATH, map_location='cpu')
    model.load_state_dict(loaded_state_dict)
    print("모델 가중치 로드 완료.")
else:
    print("새로운 모델 초기화 완료. 저장된 가중치를 찾을 수 없습니다.")

model.to(device)

num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Custom Bert 모델 초기화 완료. 총 학습 가능 파라미터 수 : {num_params}')
print(f'모델이 담긴 장치 : {device}')

EPOCHS = 3
LEARNING_RATE = 5e-8
WEIGHT_DECAY = 0.1
optimizer = AdamW(model.parameters(),lr=LEARNING_RATE,weight_decay=WEIGHT_DECAY)

train_losses = []
acc = 0
prev_acc = 0
cnt = 0

print(f"\n<--- 학습 시작 ---> ({EPOCHS} 에폭)")

for e in range(EPOCHS):
    loss_sum = 0
    progress_bar = tqdm(train_loader,desc=f"Train Epoch {e+1}")
    model.train()
    for step, batch in enumerate(progress_bar):
        batch = {k: v.to(device) for k, v in batch.items()}
        
        outputs = model.forward(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"],
            token_type_ids=batch["token_type_ids"],
            labels=batch["labels"]
        )
        loss = outputs["loss"]

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        loss_sum += loss.item()
        progress_bar.set_postfix({'loss':f"{(loss_sum/(step+1)):.4f}"})
        del outputs, loss
        if 'ccuda' in str(device):
            torch.cuda.empty_cache()

    avg_train_loss = loss_sum / len(train_loader)
    train_losses.append(avg_train_loss)

    print(f"Train Epoch {e+1} 완료. 평균 학습 손실 : {avg_train_loss:.4f}")

    model.eval()
    correct = 0
    total = 0
    val_progress = tqdm(val_loader, desc=f"Validation Epoch {e+1}")
    with torch.no_grad():
        for step, batch in enumerate(val_progress):
            batch = {k: v.to(device) for k, v in batch.items()}
            
            y = model.forward(input_ids=batch["input_ids"],
                attention_mask=batch["attention_mask"],
                token_type_ids=batch["token_type_ids"]
            )
            y_logits = y["logits"]
            t = batch["labels"]
            correct += (y_logits.argmax(dim=-1) == t).sum().item()
            total += len(batch["input_ids"])
            val_progress.set_postfix({"acc" : f"{((correct/total)*100):.2f}%"})
            
    acc = correct / total
    
    print(f"Validation Epoch {e+1} 완료. 검증 정확도 : {(acc*100):.2f}%")
    
    if acc <= prev_acc:
        cnt += 1
    else :
        torch.save(model.state_dict(), MODEL_SAVE_PATH)
        cnt = 0
        prev_acc = acc

    if cnt >= 5:
        print("train halted")
        break
       
print("\n<--- 학습 완료 --->")